# Complete Text-to-SQL Workflow with All 4 Agents

This notebook demonstrates a complete text-to-SQL workflow using all 4 specialized agents:
- **QueryAnalyzerAgent**: Analyzes user queries and creates query trees
- **SchemaLinkerAgent**: Links query intents to database schema
- **SQLGeneratorAgent**: Generates SQL from linked schema
- **SQLEvaluatorAgent**: Executes and evaluates SQL results

## Key Features

1. **Structured Memory Management**: Uses KeyValueMemory with specialized managers
2. **Query Tree Architecture**: All agents operate on nodes in a query tree
3. **Automatic SQL Execution**: SQLEvaluator automatically executes SQL
4. **Iterative Refinement**: Coordinator can retry if SQL is incorrect
5. **Complete Traceability**: Full visibility into the workflow process

In [1]:
import os
import sys
import asyncio
import logging
from pathlib import Path
from typing import Dict, Any, List, Optional
from dotenv import load_dotenv

sys.path.append('../src')
load_dotenv()

# Important: For running this notebook, ensure OPENAI_API_KEY is set
# You can run: source ../.env && export OPENAI_API_KEY
# Or set it here directly (not recommended for production)
if not os.getenv("OPENAI_API_KEY"):
    print("WARNING: OPENAI_API_KEY not found in environment")
    print("Run: source ../.env && export OPENAI_API_KEY")
else:
    print("OPENAI_API_KEY found in environment")

# Set up logging
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Reduce noise from autogen
logging.getLogger('autogen_core').setLevel(logging.WARNING)
logging.getLogger('httpx').setLevel(logging.WARNING)

OPENAI_API_KEY found in environment


## 1. Import All Required Components

In [2]:
# Memory and managers
from keyvalue_memory import KeyValueMemory
from task_context_manager import TaskContextManager
from query_tree_manager import QueryTreeManager
from database_schema_manager import DatabaseSchemaManager
from node_history_manager import NodeHistoryManager

# Schema reader
from schema_reader import SchemaReader

# All 4 agents
from query_analyzer_agent import QueryAnalyzerAgent
from schema_linker_agent import SchemaLinkerAgent
from sql_generator_agent import SQLGeneratorAgent
from sql_evaluator_agent import SQLEvaluatorAgent

# Memory types
from memory_content_types import (
    TaskContext, QueryNode, NodeStatus, TaskStatus,
    QueryMapping, TableMapping, ColumnMapping, JoinMapping,
    TableSchema, ColumnInfo, ExecutionResult
)

# AutoGen components
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [3]:
# Initialize shared memory
memory = KeyValueMemory()

# Initialize managers
task_manager = TaskContextManager(memory)
tree_manager = QueryTreeManager(memory)
schema_manager = DatabaseSchemaManager(memory)
history_manager = NodeHistoryManager(memory)

print("Initialized memory and managers")

Initialized memory and managers


## 3. Load Database Schema

In [4]:
# Database configuration
data_path = "/home/norman/work/text-to-sql/MAC-SQL/data/bird"
tables_json_path = Path(data_path) / "dev_tables.json"
db_name = "california_schools"

# Initialize task
task_id = "workflow_demo_001"
# Test query
test_query = "What is the highest eligible free rate for K-12 students in schools located in Alameda County?"
print(f"Processing query: {test_query}")
print("-" * 80)

await task_manager.initialize(task_id, test_query, db_name)

# Load schema using SchemaReader
schema_reader = SchemaReader(
    data_path=data_path,
    tables_json_path=str(tables_json_path),
    dataset_name="bird",
    lazy=False
)

# Load schema into memory
await schema_manager.load_from_schema_reader(schema_reader, db_name)

# Get schema summary
summary = await schema_manager.get_schema_summary()
print(f"Loaded database '{db_name}' schema:")
print(f"  Tables: {summary['table_count']}")
print(f"  Total columns: {summary['total_columns']}")
print(f"  Foreign keys: {summary['total_foreign_keys']}")

2025-05-25 12:44:53,414 - TaskContextManager - INFO - Initialized task context for task workflow_demo_001


Processing query: What is the highest eligible free rate for K-12 students in schools located in Alameda County?
--------------------------------------------------------------------------------
load json file from /home/norman/work/text-to-sql/MAC-SQL/data/bird/dev_tables.json

Loading all database info...
Found 11 databases in bird dataset


2025-05-25 12:45:05,905 - DatabaseSchemaManager - INFO - Initialized empty database schema
2025-05-25 12:45:05,906 - DatabaseSchemaManager - INFO - Added table 'frpm' to schema
2025-05-25 12:45:05,906 - DatabaseSchemaManager - INFO - Added table 'satscores' to schema
2025-05-25 12:45:05,907 - DatabaseSchemaManager - INFO - Added table 'schools' to schema
2025-05-25 12:45:05,907 - DatabaseSchemaManager - INFO - Loaded schema for database 'california_schools' with 3 tables


Loaded database 'california_schools' schema:
  Tables: 3
  Total columns: 89
  Foreign keys: 2


## 4. Initialize All 4 Agents

In [5]:
# LLM configuration
llm_config = {
    "model_name": "gpt-4o",
    "temperature": 0.1,
    "timeout": 60
}

# Initialize all agents
query_analyzer = QueryAnalyzerAgent(memory, llm_config)
schema_linker = SchemaLinkerAgent(memory, llm_config)
sql_generator = SQLGeneratorAgent(memory, llm_config)
sql_evaluator = SQLEvaluatorAgent(memory, llm_config)

print("Initialized all 4 agents:")
print("  - QueryAnalyzerAgent")
print("  - SchemaLinkerAgent")
print("  - SQLGeneratorAgent")
print("  - SQLEvaluatorAgent")

2025-05-25 12:45:05,928 - QueryAnalyzerAgent - INFO - Initialized query_analyzer with model gpt-4o
2025-05-25 12:45:05,939 - SchemaLinkerAgent - INFO - Initialized schema_linker with model gpt-4o
2025-05-25 12:45:05,950 - SQLGeneratorAgent - INFO - Initialized sql_generator with model gpt-4o
2025-05-25 12:45:05,961 - SQLEvaluatorAgent - INFO - Initialized sql_evaluator with model gpt-4o


Initialized all 4 agents:
  - QueryAnalyzerAgent
  - SchemaLinkerAgent
  - SQLGeneratorAgent
  - SQLEvaluatorAgent


## 5. Create Coordinator Agent

The coordinator orchestrates the workflow by calling each agent in sequence and handling iterations if needed.

In [6]:
# Initialize OpenAI client for coordinator
coordinator_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    temperature=0.1,
    timeout=120,
    api_key=os.getenv("OPENAI_API_KEY")
)

# Create coordinator agent
coordinator = AssistantAgent(
    name="coordinator",
    system_message="""You coordinate a text-to-SQL workflow using 4 specialized agents.

Your agents are:
1. query_analyzer - Analyzes user queries and creates query trees
2. schema_linker - Links query intent to database schema
3. sql_generator - Generates SQL from linked schema
4. sql_evaluator - Executes and evaluates SQL results

Workflow:
1. Call query_analyzer with the user's query
   - This creates a query tree and stores the node ID in memory

2. Call schema_linker with: "Link query to database schema"
   - The agent will automatically use the node ID from memory

3. Call sql_generator with: "Generate SQL query"
   - The agent will automatically use the node ID from memory

4. Call sql_evaluator with: "Analyze SQL execution results"
   - The agent will automatically use the node ID from memory

5. If the evaluator indicates issues:
   - Analyze what went wrong
   - You may need to call agents again with better guidance
   - The agents will continue using the same node ID from memory

6. Once you have correct SQL with good results, provide a final answer and say "TERMINATE"

IMPORTANT: The agents automatically track the current node ID in memory, so you don't need to specify it.""",
    model_client=coordinator_client,
    tools=[query_analyzer.get_tool(), schema_linker.get_tool(), sql_generator.get_tool(), sql_evaluator.get_tool()]
)

## 6. Define Helper Functions

### Helper to Display Full Memory Contents

In [7]:
async def display_memory_contents():
    """Display full memory contents in a structured format"""
    print("\n" + "="*80)
    print("FULL MEMORY CONTENTS")
    print("="*80)
    
    # Get all memory data
    memory_data = await memory.show_all(format="json")
    
    if not memory_data:
        print("Memory is empty")
        return
    
    for key, value in memory_data.items():
        print(f"\n[{key}]")
        print("-" * 40)
        
        if isinstance(value["value"], dict):
            # Pretty print dictionaries
            import json
            print(json.dumps(value["value"], indent=2))
        elif isinstance(value["value"], list):
            # Pretty print lists
            import json
            print(json.dumps(value["value"], indent=2))
        else:
            # Print raw value
            print(value["value"])
    
    print("\n" + "="*80)
    print(f"Total keys in memory: {len(memory_data)}")
    print("="*80)

async def display_query_tree():
    """Display the current query tree structure"""
    tree = await tree_manager.get_tree()
    if not tree or "nodes" not in tree:
        print("No query tree found")
        return
    
    print("\n" + "="*60)
    print("QUERY TREE STRUCTURE")
    print("="*60)
    
    for node_id, node_data in tree["nodes"].items():
        print(f"\nNode: {node_id}")
        print(f"  Intent: {node_data.get('intent', 'N/A')}")
        print(f"  Status: {node_data.get('status', 'N/A')}")
        
        # Show mapping if available
        if 'mapping' in node_data and node_data['mapping']:
            mapping = node_data['mapping']
            if mapping.get('tables'):
                tables = [t['name'] for t in mapping['tables']]
                print(f"  Tables: {', '.join(tables)}")
            if mapping.get('columns'):
                cols = [f"{c['table']}.{c['column']}" for c in mapping['columns']]
                print(f"  Columns: {', '.join(cols[:3])}..." if len(cols) > 3 else f"  Columns: {', '.join(cols)}")
        
        # Show SQL if available
        if 'sql' in node_data and node_data['sql']:
            sql_preview = node_data['sql'].strip().replace('\n', ' ')[:100]
            print(f"  SQL: {sql_preview}..." if len(sql_preview) == 100 else f"  SQL: {sql_preview}")
        
        # Show execution result if available
        if 'executionResult' in node_data and node_data['executionResult']:
            result = node_data['executionResult']
            print(f"  Execution: {result.get('rowCount', 0)} rows")
            if result.get('error'):
                print(f"  Error: {result['error']}")

async def display_final_results():
    """Display the final SQL and execution results"""
    tree = await tree_manager.get_tree()
    if not tree or "nodes" not in tree:
        print("No results found")
        return
    
    print("\n" + "="*60)
    print("FINAL RESULTS")
    print("="*60)
    
    # Find nodes with SQL
    for node_id, node_data in tree["nodes"].items():
        if 'sql' in node_data and node_data['sql']:
            print(f"\nNode: {node_id}")
            print(f"Intent: {node_data.get('intent', 'N/A')}")
            print(f"\nSQL:\n{node_data['sql']}")
            
            if 'executionResult' in node_data and node_data['executionResult']:
                result = node_data['executionResult']
                print(f"\nExecution Result:")
                print(f"  Rows returned: {result.get('rowCount', 0)}")
                
                if result.get('data') and len(result['data']) > 0:
                    print(f"\nSample data (first 5 rows):")
                    for i, row in enumerate(result['data'][:5]):
                        print(f"  {row}")
                
                # Check for analysis
                analysis_key = f"node_{node_id}_analysis"
                analysis = await memory.get(analysis_key)
                if analysis:
                    print(f"\nEvaluation:")
                    print(f"  Answers intent: {analysis.get('answers_intent', 'N/A')}")
                    print(f"  Result quality: {analysis.get('result_quality', 'N/A')}")
                    print(f"  Summary: {analysis.get('result_summary', 'N/A')}")

## 7. Test with a Simple Query

In [8]:
# Create a team with termination condition
termination_condition = TextMentionTermination("TERMINATE")
team = RoundRobinGroupChat(
    participants=[coordinator],
    termination_condition=termination_condition
)

# Run the workflow
stream = team.run_stream(task=test_query)

In [9]:
# Stream and display messages
message_count = 0
async for message in stream:
    message_count += 1
    # Only show coordinator messages to reduce noise
    if hasattr(message, 'source') and message.source == 'coordinator':
        print(f"\n[Step {message_count}] Coordinator:")
        if hasattr(message, 'content'):
            if isinstance(message.content, str):
                print(message.content)
            elif isinstance(message.content, list) and len(message.content) > 0:
                # Tool call
                for tool_call in message.content:
                    if hasattr(tool_call, 'name'):
                        print(f"  → Calling {tool_call.name}")

print("\n" + "="*80)
print("WORKFLOW COMPLETE")
print("="*80)


[Step 2] Coordinator:
  → Calling query_analyzer


2025-05-25 12:45:08,487 - QueryTreeManager - INFO - Initialized query tree with root node node_1748191508.487403_root
2025-05-25 12:45:08,487 - NodeHistoryManager - INFO - Added create operation for node node_1748191508.487403_root
2025-05-25 12:45:08,487 - QueryAnalyzerAgent - INFO - Simple query - set root node_1748191508.487403_root as current node
2025-05-25 12:45:08,488 - QueryAnalyzerAgent - INFO - Query analysis completed. Complexity: simple. Root node: node_1748191508.487403_root



[Step 3] Coordinator:
  → Calling query_analyzer

[Step 4] Coordinator:
{"messages": [{"source": "user", "models_usage": null, "metadata": {}}, {"source": "query_analyzer", "models_usage": {"prompt_tokens": 3165, "completion_tokens": 108}, "metadata": {}}], "stop_reason": null}

[Step 5] Coordinator:
  → Calling schema_linker


2025-05-25 12:45:12,785 - QueryTreeManager - INFO - Updated node node_1748191508.487403_root
2025-05-25 12:45:12,786 - NodeHistoryManager - INFO - Added revise operation for node node_1748191508.487403_root
2025-05-25 12:45:12,786 - SchemaLinkerAgent - INFO - Updated node node_1748191508.487403_root with schema mapping



[Step 6] Coordinator:
  → Calling schema_linker

[Step 7] Coordinator:
{"messages": [{"source": "user", "models_usage": null, "metadata": {}}, {"source": "schema_linker", "models_usage": {"prompt_tokens": 5012, "completion_tokens": 215}, "metadata": {}}], "stop_reason": null}

[Step 8] Coordinator:
  → Calling sql_generator


...


[Step 9] Coordinator:
  → Calling sql_generator

[Step 10] Coordinator:
{"messages": [{"source": "user", "models_usage": null, "metadata": {}}, {"source": "sql_generator", "models_usage": {"prompt_tokens": 380, "completion_tokens": 190}, "metadata": {}}], "stop_reason": null}


2025-05-25 12:45:17,598 - SQLEvaluatorAgent - INFO - Using current node from memory: node_1748191508.487403_root
2025-05-25 12:45:17,599 - SQLEvaluatorAgent - INFO - Executing SQL for node node_1748191508.487403_root on database california_schools
2025-05-25 12:45:17,601 - QueryTreeManager - INFO - Updated node node_1748191508.487403_root


[SQLExecutor] Connecting to database: /home/norman/work/text-to-sql/MAC-SQL/data/bird/dev_databases/california_schools/california_schools.sqlite

[Step 11] Coordinator:
  → Calling sql_evaluator


2025-05-25 12:45:20,670 - SQLEvaluatorAgent - INFO - Stored analysis for node node_1748191508.487403_root - Answers intent: yes, Quality: excellent
2025-05-25 12:45:20,671 - SQLEvaluatorAgent - INFO - Root node node_1748191508.487403_root and all descendants are good - workflow complete
2025-05-25 12:45:20,671 - SQLEvaluatorAgent - INFO - Analysis complete - Answers intent: yes, Quality: excellent



[Step 12] Coordinator:
  → Calling sql_evaluator

[Step 13] Coordinator:
{"messages": [{"source": "user", "models_usage": null, "metadata": {}}, {"source": "sql_evaluator", "models_usage": {"prompt_tokens": 402, "completion_tokens": 189}, "metadata": {}}], "stop_reason": null}

[Step 14] Coordinator:
The SQL execution results indicate that the highest eligible free rate for K-12 students in schools located in Alameda County is 85%. 

TERMINATE

WORKFLOW COMPLETE


## 8. Display Query Tree and Results

In [10]:
# Display the query tree structure
await display_query_tree()


QUERY TREE STRUCTURE

Node: node_1748191508.487403_root
  Intent: Find the highest percentage of students eligible for free meals in K-12 schools located in Alameda County.
  Status: executed_success
  Tables: frpm
  Columns: frpm.County Name, frpm.Percent (%) Eligible Free (K-12)
  SQL: SELECT MAX(f."Percent (%) Eligible Free (K-12)") AS max_percentage FROM frpm AS f WHERE f."County Na...
  Execution: 1 rows


## 9. Display Full Memory Contents

This shows all data stored in memory after the workflow completes.

In [11]:
# Display full memory contents
await display_memory_contents()


FULL MEMORY CONTENTS

[workflow_complete]
----------------------------------------
True

[node_node_1748191508.487403_root_analysis]
----------------------------------------
{
  "answers_intent": "yes",
  "result_quality": "excellent",
  "result_summary": "The result shows the highest percentage of students eligible for free meals in K-12 schools in Alameda County, which is 100%.",
  "confidence_score": 0.95,
  "issues": [
    {
      "type": "data_quality",
      "description": "The result indicates that there is at least one school in Alameda County where 100% of students are eligible for free meals, which might be unusual and worth verifying.",
      "severity": "medium"
    }
  ],
  "suggestions": [
    "Verify the data source to ensure that the 100% eligibility is accurate and not due to data entry errors."
  ]
}

[execution_analysis]
----------------------------------------
{
  "answers_intent": "yes",
  "result_quality": "excellent",
  "result_summary": "The result shows the hi